**ASSIGNMENT**
- Multiprogramming on Python
- Python Threading Library
- Python Multiprocessing Library

# Question 1. (3 marks)

**Examine the code you have typed in, and explain how this code works. That is, why it produces two threads that print once per second and once every two seconds.**

## Answer for Q1:

In [ ]:
# t1.py

import threading                                   
import time

x = 1

# First thread. We pass in one parameter
def thread1(param):
    global x
    while True:
        x = x + 1
        print("This is thread 1. x is %d" % x)
        time.sleep(param)
        
# Second thread. We pass in two parameters
def thread2(param1, param2):
    global x
    while True:
        x = x * 2
        print("This is thread 2. Param 2 is %d. x is %d" % (param2, x))
        time.sleep(param1)
        
# Create the first thread
th1 = threading.Thread(target = thread1, args = [1, ])
th1.start()

# Create the second thread
th2 = threading.Thread(target = thread2, args = [2, 123])
th2.start()

**How the code works:**

Overall, the code in `t1.py` creates two separate threads that operate on a shared variable `x`: 

- The first thread `thread1` increments `x` by one, then prints the updated value of `x`, and then sleeps for `param` seconds, in which was passed as `1` when `th1` was instantiated.

- The second thread `thread2` doubles `x`, then prints the parameter `param2` (passed as `123`) and the updated value of `x`, and then sleeps for `param1` seconds (passed as `2`). `param1` and `param2` were passed as parameters when `th2` was instantiated.

Both threads run continuously due to the infinite `while` loop set to `True` without a break condition. The `threading` library helps to create an environment where two threads _appear_ to run concurrently at the same time. The `sleep()` function from `time` library is used to pause the execution of the current thread for a certain time (number of seconds) before continuing that thread's operation, during this time the thread release CPU resources which allows the other thread to run.

**Why it produces two threads that print once per second and once every two seconds:**

```python
# Create the first thread
th1 = threading.Thread(target = thread1, args = [1, ])
th1.start()
```
=> Thread `th1` is created by instantiating a `Thread` object from `threading` module, targeting the `thread1(param)` function with an argument list of `[1, ]`. Since `param` is set to `1`, `time.sleep(param)` will pause for **one second** the execution of `thread1` after each print statement before continue the loop. Therefore this thread approximately **prints once per second**, as it waits for one second between each loop iteration.


```python
# Create the second thread
th2 = threading.Thread(target = thread2, args = [2, 123])
th2.start()
```

=> Similarly, thread `th2` is created by instantiating a `Thread` object from `threading` module, targeting the `thread2(param1, param2)` function with an argument list of `[2, 123]`. Here `param1` is set to `2`, `time.sleep(param1)` will pause for **two seconds** the execution of `thread2` after each print statement. This causes the thread to approximately **print once every two seconds** because it has a two-second pause between each loop iteration.

**Inconsistency in Thread Execution**

Hereunder are few screenshots of the outputs when `t1.py` was run in different environments:

- Different outputs of `t1.py` when run in VSCode on MacOS (`CPython`):
    - ![](img/q1.1.png)
    - ![](img/q1.2.png)


- Different outputs of `t1.py` when run on Jupyter Lab on MacOS (`IPython`):
    - ![](img/q1.3.png)

From the screenshots above,the outputs vary with each execution, indicating a race condition affecting the shared variable `x`. Results from CPython in VSCode appear to have fewer variations, potentially due to the reduced overhead compared to IPython. Since Jupyter Lab cells run within an IPython kernel (which is built on top of CPython with extra layers), thread scheduling might be influenced by background tasks and the interactive nature of the environment. Meanwhile, executions in VSCode operate in a  standard CPython environment, the terminal outputs vary less than in Jupyter Lab and may result in a more predictable thread interleaving.

The Global Interpreter Lock (GIL) in CPython is designed to manage thread execution, allowing only one thread to execute at a time within a single process. However, despite the presence of the GIL, both environments still display inconsistent results due to the lack of synchronized access to the shared resource in a multi-threaded context. Potential solutions to prevent the race condition could be to implement a threading lock or other synchronization techniques to manage the shared state.

In summary, `t1.py` uses `threading` to run two threads within the same process and shared memory space, meaning variable `x` is shared between threads, leading to mixed sequence of incremented and doubled output values. Sharing the same global variable as in `t1.py` can lead to inconsistent results and potential race conditions due to interleaved execution and the lack of a synchronization mechanism. 


# Question 2

## Question 2a. (2 marks)

**Run the code with the following values of “n”, recording whether you get the correct result**:

```
n = 10, Correct: Y / N
n = 1000, Correct: Y / N
n = 1000000, Correct: Y / N
n = 10000000, Correct: Y / N
```

In [ ]:
# t2.py

import threading

# The number to add.
# Change this for your experiment
n = 100_000

# Result variable
result = 0

def thread1(count):
    global result

    x = 0
    for i in range(1, count + 1):
        x = x + i
        result = x

        
def thread2():
    global result
    result = result * 3

th1 = threading.Thread(target=thread1, args=[n, ])
th2 = threading.Thread(target = thread2)

th1.start()
th2.start()

correct = n * (n + 1) / 2 * 3
print("Correct result is %d." % correct)
print("Final result is %d." % result)

if result == correct:
    print("CORRECT!")
else:
    print("WRONG")


### Answer for Q2a:

In general, `t2.py` attempts to calculate sum of all numbers from 1 to `n` and then multiply by `3` using threading.

Hereunder are some few sample outputs from multiple executions of `t2.py` with different values of `n`:

![](img/q2-1.png)

Overall, the results' pattern seems to be summed up as following. However this pattern is not guaranteed:
```
n = 10, Correct: Y
n = 1000, Correct: Y
n = 1000000, Correct: N
n = 10000000, Correct: N
```

## Question 2b. (4 marks)
**Some of the answers in 2a are incorrect. Explain why you get incorrect results.**

### Answer for Q2b:

The executions show inconsistent results, especially for large values of `n`. 

When `n` is relatively small, the results shows more correct results, this might be due to several reasons such as caching or CPU pipelining, thread starting time and scheduling, etc., For instance, the shorter execution time in which `thread1` might have finished changing the shared `result` variable before `thread2` finished its execution, which then decreases the likelihood of conflict. When `n` is large, the time for `thread1` to execute all of its loop iterations might be significantly longer, which then leads to situation where both `thread1` and `thread2` trying to access and modify the shared `result` variable at the same time. 

**Potential issues**

The results of the thread executions with different values of `n` show that as value `n` increases, there is also a higher chance of encountering incorrect results, as both `thread1` and `thread2` trying to enter critical section to access the shared resource simultaneously, which leads to unpredictability in the outputs. Eventhough Python's Global Interpreter Lock (GIL) ensures that only one thread can execute at a time, however, context switching might still happen when another thread is waiting. This pattern is indicative of a race condition (main reason for inconsistent outputs of `t2.py`) where the program does not handle the synchronization of threads accessing and modifying the shared `result` variable. Other contributing reasons might include MacOS's thread scheduler, thread preemption, real-time scheduling policies, etc., however, these factors only worsen the situation where race condition happens without proper synchronization mechanism. 

**Potential solutions**

In order to fix this concurrency problem, a suitable synchronization mechanism should be employed to ensure that `thread2` does not start multiplying `result` until `thread1` has finished adding the numbers. This can be achieved through multiple ways, such as adopting semaphores (access control through the use of a counter) or mutexes (threading lock to ensure mutual exlusion) through `threading.Lock()` or `threading.Semaphore()` in Python. Monitors with conditional variables could also solve this problem, where `thread2` waits on a condition variable until `thread1` gives signal that it has finished. Another solution could be using barrier, in which `thread2` would only process if `thread1` has passed the barrier. Furthermore, thread safe data structures such as Python's `queue` module with First-In-First-Out (FIFO) implementation can also ensure the order of thread execution where `thread2` only retrieves `result` after `thread1` has finished. 

The following table is a summary of potential synchronization solutions to handle the race condition in `t2.py`. However, depending on the OS/Device and different scenarios, other mechanisms not listed here could be more beneficial to their specific use case and scope.

![](img/q2-2.png)

# Question 3. (4 marks)
**t3.py now shows the correct result even for large n. Explain, using the idea of queues, how this program works to give the correct result. You may Google for what queues are.**

In [ ]:
# t3.py

import threading
from queue import Queue

resultQ = Queue()
finalQ = Queue()

# The number to add.
# Change this for your experiment
n = 10

def thread1(count):
    global resultQ
    x = 0
    for i in range(1, count + 1):
        x = x + i 
    
    resultQ.put(x)

def thread2():
    global resultQ
    global finalQ
    result = resultQ.get()
    result = result * 3
    finalQ.put(result)

th1 = threading.Thread(target = thread1, args=[n, ])
th2 = threading.Thread(target = thread2)

th1.start()
th2.start()

correct = n * (n + 1) / 2 * 3 
print("Correct result is %d." % correct)

finalResult = finalQ.get()
print("Final result is %d." % finalResult)

if finalResult == correct:
    print("CORRECT!")
else:
     print("WRONG!")

## Answer for Q3:

Both `t2.py` and `t3.py` use threading to calculate the sum of numbers from 1 to `n` and then multiply the sum by 3. The key difference is that `t3.py` uses queues to handle data sharing between threads, which ensures that the threads access and modify shared data in a thread-safe manner, leading to consistent and correct results.

The following diagram gives an overview visualization of how `t3.py` works:

![](img/q3.1.png)

The `Queue` in Python is basically a sequential collection of data or linear data structure, in which order of data elements is strictly First In First Out (FIFO). 

**Main `Queue` operations used in `t3.py`**:
- Enqueue (`put()`): Insert an element to the end of the queue.
- Dequeue (`get()`): Remove and return an element from the start of the queue.

**Main flow of `t3.py`**:

- Initialization by the main thread:
    - Sets `n` as the number up to which the sum is calculated.
    
    - Initializes two queues, `resultQ` for the sum and `finalQ` for the tripled result.


- Execution of `thread1`:
    - Computes the sum of numbers up to `n`.
    
    - Places the sum in `resultQ` using `resultQ.put(sum)` (an **enqueue** operation).


- Execution of `thread2`:
    - Waits for the sum to be available in `resultQ`.
    
    - Retrieves the sum with `resultQ.get()` (a **dequeue** operation).
    
    - Multiplies the sum by 3 and enqueues the result into `finalQ` (an **enqueue** operation).


- Result retrieval and verification by the main thread:
    - Calculates the expected correct result using a mathematical formula.
    
    - Retrieves the final result from `finalQ` with `finalQ.get()` (a **dequeue** operation) and prints it.
    
    - Compares the retrieved result with the expected result and prints whether they match.

**Key difference between `t2.py` and `t3.py`**:

| Aspect | `t2.py` | `t3.py` |
|--------|---------|---------|
| **Synchronization mechanism** | None. Direct access to global variable | Thread-safe queues (`Queue`) used to pass data between threads |
| **Race condition** | Present. `thread2` can access `result` before `thread1` completes its task | Not present. `thread2` starts after `thread1` puts the result in `resultQ` |
| **Execution order** | Not controlled. Threads execute concurrently with no enforced order | Enforced order. `thread2` waits for `thread1` to complete via queue |
| **Data sharing** | Global variable `result` accessed by both threads | Each thread has its own local data until they put or get from the queues |
| **Thread communication** | Implicit. Based on shared global state | Explicit. Through the `put()` and `get()` methods of the queues |
| **Safety of operation** | Not safe. Concurrent modifications without locks | Safe. Queues handle internal locking mechanisms |
| **Result consistency** | Inconsistent and often incorrect due to race condition | Consistent and correct as operations are properly sequenced |
| **Blocking operations** | No blocking. Threads run without waiting for each other | Blocking `get()` method used to wait for the queue to be filled |
| **Debugging difficulty** | High. Race conditions are hard to replicate and debug | Lower. Controlled execution flow and no shared state modifications |

The usage of queues in `t3.py` is essential for synchronization. The queues act like a FIFO buffer, which is conceptually similar to the FCFS (First-Come, First-Served) scheduling algorithm. 

In `t3.py`, the `Queue` module provides thread safety by using internal locks that ensure that only one thread can access a queue at a time. This is alike to synchronization hardware where atomic instructions prevent race conditions. The thread-safe `Queue` class also ensures that elements are processed in the exact order they are added. 

In the context of `t3.py`, `queue.get()` serves as a blocking mechanism, ensuring sequential execution where necessary. For example, this means that `thread2` will always wait for `thread1` to complete its task and place the sum in `resultQ` before proceeding with its multiplication task, thus preventing any race conditions.

`t3.py` demonstrates how thread-safe queues and enforced sequential access can effectively be used to implement a producer-consumer scenario where `thread1` (the producer) safely passes data to `thread2` (the consumer) without any race conditions, ensuring the final result is always correct.

# Question 4. (2 marks)

In [ ]:
# p1.py

import multiprocessing
import time

x = 1

# First thread. We pass in one parameter
def process1(param):
    global x
    while True:
        x = x + 1
        print("This is process 1. x is %d" % x)
        time.sleep(param)

# Second process. We pass in two parameters
def process2(param1, param2):
    global x
    while True:
        x = x * 2
        print("This is process 2. Param 2 is %d. x is %d" % (param2, x))
        time.sleep(param1)

def main():
    # Create the first thread
    p1 = multiprocessing.Process(target=process1, args=[1, ])
    p1.start()
    p2 = multiprocessing.Process(target=process2, args=[2, 123])
    p2.start()

if __name__ == "__main__":
    main()


**Since p1.py and t1.py are essentially the same program, they should produce the same results, but do not. Explain why.**

## Answer for Q4:

Some samples of terminal outputs of `p1.py`:

![](img/q4.1.png)
![](img/q4.2.png)

Both `t1.py` and `p1.py` aim to concurrently modify the shared variable `x` through addition and multiplication operations. Syntactically they are similar, in which they share majority of the structure and code layout with the identical logic flow. The main syntactical differences are caused by using different concurrency modules as `threading` in `t1.py` and `multiprocessing` in `p1.py`, which reflect how each program approaches concurrency differently that eventually leads to different terminal outputs.



The key differences between `t1.py` and `p1.py` are highlighted as following:

| Aspect                       | `t1.py` with Threading                     | `p1.py` with Multiprocessing                 |
|------------------------------|--------------------------------------------|---------------------------------------------|
| **Concurrency model**        | Threads                                   | Processes                                   |
| **Module imported**              | `threading`                               | `multiprocessing`                           |
| **Global variable sharing**  | Shared among threads                      | Not shared; each process has its own copy   |
| **Modification of `x`**      | `x` is modified by all threads            | `x` is modified by each process independently|
| **Synchronization mechanism**| Not present; prone to race conditions      | Not necessary; processes are independent    |
| **Output pattern**           | Interleaved and dependent on thread execution order | Parallel and consistent within each process |
| **Memory space**             | Shared among threads                       | Independent for each process                |
| **Execution**                | Concurrent execution within a single Python interpreter | Parallel execution in separate interpreters |
| **Result consistency**       | Inconsistent due to lack of synchronization | Consistent as each process operates independently |
| **Process/Thread Safety**    | Unsafe without explicit locks or synchronization | Safe as each process operates in a separate memory space |
| **Implications**             | Potential for concurrent access issues; requires careful management of shared data | More overhead but better isolation; eliminates most concurrent access issues |

`t1.py` uses `threading` to run two threads within the same process and shared memory space, meaning variable `x` is shared between threads, leading to mixed sequence of incremented and doubled output values. Sharing the same global variable as in `t1.py` can lead to inconsistent results and potential race conditions due to interleaved execution and the lack of a synchronization mechanism. 

While `p1.py` uses `multiprocessing` to create two separate processes, each with isolated memory space with its own copy of global variable `x`, allowing for true parallel execution across multiple CPU cores, resulting in two separate output sequences with consisten results: one of incremented values and one of doubled values.

However, for `p1.py`, the processes' output in to the terminal can still occur in an unpredictable sequence due to the operating system's process scheduling. Even though processes can run in parallel, without explicit management of their execution order, the printed outputs may not follow a consistent order, and one process might output several times before another starts, similar to the threading scenario discussed in Question 1 about `t1.py`.

# Question 5. (5 marks)

You can use queues to fix `p1.py` so that it produces the same result as `t1.py`. To do this:
    a. Change `process1` so that it increments `x` by 1 and sends it over to `process2`. It then waits for the result from `process2`.
    b. Change `process2` so that it waits for the result from `process1`, multiplies it by 2 then sends it back to `process1`.

IMPORTANT NOTE: For this to work correctly, you must use the `multiprocessing` version of `Queue`. So within your code, import `Queue` from `multiprocessing` instead of from `queue`. That is:

DO NOT DO THIS:
```python
from queue import Queue
```
DO THIS INSTEAD:
```python
from multiprocessing import Queue`
```

**Detail the changes you made to p1.py to make it give the same results at t1.py below and explain why it works.**

## Answer for Q5:

`t1.py` has an inherent inconsistency due to the Python's Global Interpreter Lock and the scheduler's management of threads. When using Python's `multiprocessing` module, which process gets executed first will be determined by the OS's scheduling, which can introduce certain randomness or concurrent, especially on a multi-core processor where true parallelism can occur. Therefore, a similar level of unpredictability in the execution of the processes can also be simulated in `modified-p1-1.py`.

The `modified-p1-1.py` below is the modified version of `p1.py` to simulate the behaviour of `t1.py`.


In [ ]:
# modified-p1-1.py

import multiprocessing
import time

def process1(q, param):
    while True:
        x = q.get()  
        x = x + 1
        print("This is process 1. x is %d" % x)
        q.put(x)
        time.sleep(param)

def process2(q, param1, param2):
    while True:
        x = q.get()
        x = x * 2
        print("This is process 2. Param 2 is %d. x is %d" % (param2, x))
        q.put(x)
        time.sleep(param1)

def main():
    q = multiprocessing.Queue()

    q.put(1)

    p1 = multiprocessing.Process(target=process1, args=[q, 1])
    p2 = multiprocessing.Process(target=process2, args=[q, 2, 123])

    p1.start()
    p2.start()

    p1.join()
    p2.join()

if __name__ == "__main__":
    main()


Sample terminal outputs from multiple executions of `modified-p1-1.py`:

![](img/q5-1.png)
![](img/q5-2.png)
![](img/q5-3.png)

Key changes made:

| Aspect             | p1.py                                    | modified-p1-1.py                           |                                                                                                                                          
|--------------------|------------------------------------------|--------------------------------------------|
| **Global variable**    | `x = 1` defined globally                | Global `x` is removed                     |                                       
| **Process functions**  | `process1` and `process2` use `global x` | `process1` and `process2` use a Queue `q` |                                             
| **Parameter handling** | `process1` takes one parameter, `process2` takes two | Both functions take an additional parameter `q`, which is the Queue instance |
| **Main function**      | Processes `p1` and `p2` are started without initializing `x` | `main` initializes a Queue `q`, uses `q.put(1)` to set the initial `x` value, and then starts `p1` and `p2` | 
| **Starting value of `x`** | `x` is initialized to `1` as a global variable | `q.put(1)` is used to simulate the initial setting of `x` to `1` | 
| **Process execution**  | No synchronization in accessing `x`     | Synchronization is achieved using `q.get()` and `q.put()` | 
| **Termination**        | Processes run indefinitely without main waiting for them | `main` waits for process termination with `join()` |

The key change is the replacement of the global variable `x` with a multiprocessing `Queue` to manage the shared state between processes. The initial value of `x`, previously set as global variable, which then be copied into individual process' separate space, now is initialized by using `q.put(1)`, which places the value into the shared `q`. Both `process1` and `process2` are now accepting `q` as a parameter, enabling them to retrieve and update the value of `x`, ensuring safe inter-process communication. The modified code ensures that access to `x` is synchronized between processes, using the Queue's `q.get()` and `q.put()` methods, which are inherently thread-safe and prevent race conditions.

Since the `Queue` ensures that processes access the shared variable `x` one at a time, it is harder to simulate the race condition that happened in `t1.py`. Furthermore, bypassing the synchronization mechanism provided by the `Queue` would defeat the purpose of using `Queue`.